In [1]:
import cpa
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad

[rank: 0] Global seed set to 0
2024-09-10 09:13:53,185	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-09-10 09:13:56,367	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_2.h5ad"
adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_2.h5ad"
adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_2.h5ad"

In [3]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [4]:
try:
    data_path = 'combo_sciplex_prep_hvg_filtered.h5ad'
    adata = sc.read(data_path)
except:
    import gdown
    gdown.download('https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t')
    data_path = 'combo_sciplex_prep_hvg_filtered.h5ad'
    adata = sc.read(data_path)

adata

adata.obs["index"] = adata.obs_names.values
adata.obs["index_adapted"] = adata.obs.apply(lambda x: x["index"][:-2], axis=1)
adata.obs.set_index("index_adapted", inplace=True)

adata_ood.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_ood.obs["condition_ID"] = adata.obs["condition_ID"]
adata_ood.obs["log_dose"] = adata.obs["log_dose"]
adata_ood.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_ood.obs["cell_type"] = adata.obs["cell_type"]

adata_test.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_test.obs["condition_ID"] = adata.obs["condition_ID"]
adata_test.obs["log_dose"] = adata.obs["log_dose"]
adata_test.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_test.obs["cell_type"] = adata.obs["cell_type"]

adata_train.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_train.obs["condition_ID"] = adata.obs["condition_ID"]
adata_train.obs["log_dose"] = adata.obs["log_dose"]
adata_train.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_train.obs["cell_type"] = adata.obs["cell_type"]

In [5]:
control_cells_gex = adata_ood[adata_ood.obs["condition"]=="control"].X.A
obs_names = list(adata_ood[adata_ood.obs["condition"]=="control"].obs_names)

all_data = []
obs_df = pd.DataFrame(columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
indices = []

for condition in adata_ood.obs["condition"].cat.categories:
    if condition=="control":
        continue
    all_data.append(control_cells_gex)
    filtered_row = pd.Series(adata_ood[adata_ood.obs["condition"] == condition].obs[["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"]].iloc[0].values)
    filtered_row_df = pd.DataFrame([filtered_row.values], columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
    duplicated_rows = pd.concat([filtered_row_df] * 500, ignore_index=True)
    obs_df = pd.concat([obs_df, duplicated_rows], ignore_index=True)

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)


# Create the Anndata object
adata_ood_new = ad.AnnData(X=all_data_array, obs=obs_df)
adata_ood_new.var = adata_ood.var


control_cells_gex = adata_test[adata_test.obs["condition"]=="control"].X.A
obs_names = list(adata_test[adata_test.obs["condition"]=="control"].obs_names)

all_data = []
obs_df = pd.DataFrame(columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
indices = []

for condition in adata_test.obs["condition"].cat.categories:
    if condition=="control":
        continue
    all_data.append(control_cells_gex)
    filtered_row = pd.Series(adata_test[adata_test.obs["condition"] == condition].obs[["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"]].iloc[0].values)
    filtered_row_df = pd.DataFrame([filtered_row.values], columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
    duplicated_rows = pd.concat([filtered_row_df] * 500, ignore_index=True)
    obs_df = pd.concat([obs_df, duplicated_rows], ignore_index=True)

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)


# Create the Anndata object
adata_test_new = ad.AnnData(X=all_data_array, obs=obs_df)
adata_test_new.var = adata_test.var

In [6]:
import anndata

adata_combined = anndata.concat((adata_train, adata_test_new, adata_ood_new), label="split", keys=["train", "test", "ood"], join="inner")

In [7]:
frac_valid = adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs / (
    adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs + adata[adata.obs["split_1ct_MEC"] == "train"].n_obs
)

In [8]:
import numpy as np


def create_split(x):
    if x["split"] != "train":
        return x["split"]
    is_train = np.random.choice(2, p=[frac_valid, 1 - frac_valid])
    if is_train:
        return "train_train"
    return "train_valid"


adata_combined.obs["split_1ct_MEC"] = adata_combined.obs.apply(create_split, axis=1)

In [9]:
cpa.CPA.setup_anndata(adata_combined,
                      perturbation_key='condition_ID',
                      dosage_key='log_dose',
                      control_group='CHEMBL504',
                      batch_key=None,
                      is_count_data=False,
                      categorical_covariate_keys=['cell_type'],
                      deg_uns_cat_key='cov_drug_dose',
                      max_comb_len=2,
                     )

100%|██████████| 61467/61467 [00:00<00:00, 1143586.50it/s]
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [10]:
ae_hparams = {
    "n_latent": 128,
    "recon_loss": "gauss",
    "doser_type": "logsigm",
    "n_hidden_encoder": 512,
    "n_layers_encoder": 3,
    "n_hidden_decoder": 512,
    "n_layers_decoder": 3,
    "use_batch_norm_encoder": True,
    "use_layer_norm_encoder": False,
    "use_batch_norm_decoder": True,
    "use_layer_norm_decoder": False,
    "dropout_rate_encoder": 0.1,
    "dropout_rate_decoder": 0.1,
    "variational": False,
    "seed": 434,
}

trainer_params = {
    "n_epochs_kl_warmup": None,
    "n_epochs_pretrain_ae": 30,
    "n_epochs_adv_warmup": 50,
    "n_epochs_mixup_warmup": 3,
    "mixup_alpha": 0.1,
    "adv_steps": 2,
    "n_hidden_adv": 64,
    "n_layers_adv": 2,
    "use_batch_norm_adv": True,
    "use_layer_norm_adv": False,
    "dropout_rate_adv": 0.3,
    "reg_adv": 20.0,
    "pen_adv": 20.0,
    "lr": 0.0003,
    "wd": 4e-07,
    "adv_lr": 0.0003,
    "adv_wd": 4e-07,
    "adv_loss": "cce",
    "doser_lr": 0.0003,
    "doser_wd": 4e-07,
    "do_clip_grad": False,
    "gradient_clip_value": 1.0,
    "step_size_lr": 45,
}

In [11]:
model = cpa.CPA(
    adata=adata_combined,
    split_key="split_1ct_MEC",
    train_split="train_train",
    valid_split="train_valid",
    **ae_hparams,
)

[rank: 0] Global seed set to 434


In [12]:
model.train(
    max_epochs=2000,
    use_gpu=True,
    batch_size=128,
    plan_kwargs=trainer_params,
    early_stopping_patience=10,
    check_val_every_n_epoch=5,
    save_path="/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/split_2",
)

100%|██████████| 32/32 [00:00<00:00, 238.11it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 5/2000:   0%|          | 4/2000 [01:02<7:43:07, 13.92s/it, v_num=1, recon=-2.54e+3, r2_mean=0.794, adv_loss=2.46, acc_pert=0.168]


Epoch 00004: cpa_metric reached. Module best state updated.


Epoch 10/2000:   0%|          | 9/2000 [02:05<6:49:19, 12.34s/it, v_num=1, recon=-3.05e+3, r2_mean=0.799, adv_loss=2.35, acc_pert=0.183, val_recon=-2.84e+3, disnt_basal=0.073, disnt_after=0.096, val_r2_mean=0.796, val_KL=nan]


Epoch 00009: cpa_metric reached. Module best state updated.



disnt_basal = 0.06896364603137835
disnt_after = 0.08877942658778122
val_r2_mean = 0.8042570414623231
val_r2_var = 0.46646872086311103
Epoch 15/2000:   1%|          | 14/2000 [03:05<6:32:27, 11.86s/it, v_num=1, recon=-3.25e+3, r2_mean=0.805, adv_loss=2.3, acc_pert=0.195, val_recon=-3.07e+3, disnt_basal=0.069, disnt_after=0.0888, val_r2_mean=0.804, val_KL=nan] 


Epoch 00014: cpa_metric reached. Module best state updated.


Epoch 20/2000:   1%|          | 19/2000 [04:04<6:27:14, 11.73s/it, v_num=1, recon=-3.41e+3, r2_mean=0.809, adv_loss=2.22, acc_pert=0.209, val_recon=-3.23e+3, disnt_basal=0.0678, disnt_after=0.0841, val_r2_mean=0.809, val_KL=nan]
disnt_basal = 0.06314412523657462
disnt_after = 0.07882862547866797
val_r2_mean = 0.8089734031369267
val_r2_var = 0.4644575246617448
Epoch 30/2000:   1%|▏         | 29/2000 [06:05<6:29:50, 11.87s/it, v_num=1, recon=-3.61e+3, r2_mean=0.809, adv_loss=2.24, acc_pert=0.206, val_recon=-3.4e+3, disnt_basal=0.0596, disnt_after=0.0746, val_r2_mean=0.81, val_KL=nan]  
disnt_basal = 0.056485392258218345
disnt_after = 0.06958528571499586
val_r2_mean = 0.8092238075228657
val_r2_var = 0.4652751619352156
Epoch 40/2000:   2%|▏         | 39/2000 [07:59<6:01:00, 11.05s/it, v_num=1, recon=-3.7e+3, r2_mean=0.81, adv_loss=2.31, acc_pert=0.198, val_recon=-3.45e+3, disnt_basal=0.0568, disnt_after=0.0693, val_r2_mean=0.812, val_KL=nan]  


Epoch 00039: cpa_metric reached. Module best state updated.



disnt_basal = 0.05444915743787615
disnt_after = 0.0673754594814707
val_r2_mean = 0.8150381018777725
val_r2_var = 0.47000455142000314
Epoch 50/2000:   2%|▏         | 49/2000 [09:51<5:58:13, 11.02s/it, v_num=1, recon=-3.76e+3, r2_mean=0.812, adv_loss=2.39, acc_pert=0.177, val_recon=-3.52e+3, disnt_basal=0.0538, disnt_after=0.0669, val_r2_mean=0.815, val_KL=nan]
disnt_basal = 0.05170489132258298
disnt_after = 0.06388975508173163
val_r2_mean = 0.8141948926058928
val_r2_var = 0.44964585732979445
Epoch 60/2000:   3%|▎         | 59/2000 [11:42<5:59:18, 11.11s/it, v_num=1, recon=-3.81e+3, r2_mean=0.812, adv_loss=2.53, acc_pert=0.163, val_recon=-3.52e+3, disnt_basal=0.0505, disnt_after=0.0614, val_r2_mean=0.815, val_KL=nan]
disnt_basal = 0.04816554494254255
disnt_after = 0.06020687204051344
val_r2_mean = 0.8141848026868204
val_r2_var = 0.46199582319813726
Epoch 70/2000:   3%|▎         | 69/2000 [13:33<5:49:55, 10.87s/it, v_num=1, recon=-3.85e+3, r2_mean=0.813, adv_loss=2.78, acc_pert=0.134, va

In [13]:
model.predict(adata_combined)

100%|██████████| 1921/1921 [00:06<00:00, 315.16it/s]


In [14]:
adata_combined.write(
    "/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions_2.h5ad"
)